In [ ]:
import asyncio
import nest_asyncio
import queue
nest_asyncio.apply()

# 큐 생성
# q = asyncio.Queue()  # asyncio.Queue 사용
q = queue.Queue()
# 생산자 함수
async def producer():
    for i in range(10):
        event_data = f"Event {i}"
        # await q.put(event_data)  # await 키워드 사용
        q.put(event_data)  # await 키워드 사용
        print(f"Produced: {event_data}")
        await asyncio.sleep(1)

# 소비자 함수
async def consumer():
    while True:
        data = q.get()  # await 키워드 사용
        print(f"Consumed: {data}")
        q.task_done()
        await asyncio.slee(0)

async def run():
    t1 = asyncio.create_task(producer())
    t2 = asyncio.create_task(consumer())
    await asyncio.gather(t1, t2)  # await 키워드 사용

asyncio.run(run())

Produced: Event 0
Consumed: Event 0


KeyboardInterrupt: 